In [1]:
import pandas as pd
import spacy
import pickle
from tqdm import tqdm
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'pandas'

In [6]:
!pip install pandas
!pip install spacy
!pip install tqdm
!pip install scikit-learn
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [63 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Using cached cymem-2.0.11-cp313-cp313-win_amd64.whl.metadata (8.8 kB)
        Using cached preshed-3.0.9.tar.gz (14 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached murmurhash-1.0.12-cp313-cp313-win_amd64.whl.metadata (2.2 kB)
        Using cached thinc-8.3

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl (41.0 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Skill_Gap_Predictor\venv\Scripts\python.exe: No module named spacy


In [ ]:
# ⚙️ Device check
print("📦 Loading spaCy model...")
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])  # Fast mode

# 📂 Load your dataset (change to .csv if needed)
df = pd.read_csv("UpdatedResumeDataSet.csv")  # 📝 Make sure your file has "Resume" and "Category" columns
df = df[["Category", "Resume"]].dropna()

# 🧹 Preprocess text
def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

print("🧼 Preprocessing resumes...")
tqdm.pandas()
df["Cleaned_Resume"] = df["Resume"].progress_apply(preprocess)

# 📊 TF-IDF Vectorization
print("🔠 Vectorizing...")
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df["Cleaned_Resume"])
y = df["Category"]

# 🧠 Train model
print("🧠 Training model...")
model = LogisticRegression(max_iter=1000, n_jobs=-1)
model.fit(X, y)

# 💾 Save model and vectorizer
print("💾 Saving model and vectorizer...")
with open("resume_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("resume_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("✅ Training complete! Model saved as resume_model.pkl")

In [ ]:
pip install transformers torch

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import numpy as np

# Load RoBERTa model
print("📥 Loading RoBERTa...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = RobertaModel.from_pretrained("roberta-base").to(device)

# Get embedding for a single text
def get_roberta_embedding(text):
    if not text.strip():
        return np.zeros(768)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Apply on all cleaned resumes
print("🔎 Generating RoBERTa embeddings...")
roberta_embeddings = []

for text in tqdm(df["Cleaned_Resume"], desc="Embedding Resumes"):
    roberta_embeddings.append(get_roberta_embedding(text))
    gc.collect()

df["RoBERTa_Embedding"] = roberta_embeddings

# Save to pickle file for later use
df.to_pickle("resume_roberta_embeddings.pkl")
print("✅ RoBERTa embeddings saved!")

In [ ]:
# testing code
import PyPDF2
import docx
import spacy
import pickle
import torch
import numpy as np
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
from thefuzz import fuzz

# ========== Load Models ==========
print("🔧 Loading models...")
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = RobertaModel.from_pretrained("roberta-base").to(device)

with open("resume_model.pkl", "rb") as f:
    model = pickle.load(f)
with open("resume_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# ========== Utility Functions ==========
def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])

def get_roberta_embedding(text):
    if not text.strip():
        return np.zeros(768)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def extract_text_from_file(filepath):
    if filepath.endswith(".pdf"):
        with open(filepath, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            return "\n".join([page.extract_text() or "" for page in reader.pages])
    elif filepath.endswith(".docx"):
        doc = docx.Document(filepath)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        return ""

def match_skills_fuzzy(resume_text, required_skills, threshold=80):
    matched = []
    resume_lines = resume_text.lower().splitlines()
    for skill in required_skills:
        for line in resume_lines:
            if fuzz.partial_ratio(skill.lower(), line) >= threshold:
                matched.append(skill)
                break
    return list(set(matched))

def recommend_courses(missing_skills):
    course_dict = {
        "python": "https://www.udemy.com/course/python-for-beginners/",
        "data science": "https://www.udemy.com/course/data-science-python/",
        "machine learning": "https://www.udemy.com/course/machinelearning/",
        "sql": "https://www.udemy.com/course/sql-for-data-analysis/",
        "deep learning": "https://www.udemy.com/course/deeplearning/",
        "statistics": "https://www.udemy.com/course/statistics-for-data-science/"
    }
    return {skill: course_dict.get(skill.lower(), "🔍 Course not found. Search manually.") for skill in missing_skills}

# ========== Input Section ==========
resume_path = input("📄 Enter full path to resume (PDF/DOCX): ").strip()
job_description = input("📝 Enter job description: ").strip()
required_skills_input = input("💡 Enter required skills (comma separated): ").strip()
required_skills = [skill.strip() for skill in required_skills_input.split(",") if skill.strip()]

# ========== Processing ==========
print("\n⏳ Extracting and analyzing resume...")
resume_text = extract_text_from_file(resume_path)

if not resume_text.strip():
    print("❌ Could not extract text from resume.")
else:
    # Preprocess
    cleaned_resume = preprocess(resume_text)
    cleaned_jd = preprocess(job_description)

    # Predict Job Category
    tfidf_vec = vectorizer.transform([cleaned_resume])
    predicted_category = model.predict(tfidf_vec)[0]

    # Semantic Similarity
    resume_embed = get_roberta_embedding(cleaned_resume)
    jd_embed = get_roberta_embedding(cleaned_jd)
    similarity = cosine_similarity([resume_embed], [jd_embed])[0][0]

    # Skill Matching with Fuzzy Logic
    matched_skills = match_skills_fuzzy(resume_text, required_skills, threshold=80)
    missing_skills = [skill for skill in required_skills if skill not in matched_skills]

    # Course Recommendations
    recommended_courses = recommend_courses(missing_skills)

    # Fit Score Calculation
    skill_match_score = (len(matched_skills) / len(required_skills)) * 100 if required_skills else 0
    semantic_score = round(similarity * 100, 2)
    final_fit_score = round((semantic_score * 0.6) + (skill_match_score * 0.4), 2)

    # ========== Output ==========
    print("\n✅ Final Results:")
    print(f"📌 Predicted Job Category: {predicted_category}")
    print(f"🔗 Semantic Similarity Score (Resume ↔ JD): {semantic_score}%")
    print(f"🎯 Skill Match Score: {round(skill_match_score, 2)}%")
    print(f"🧠 Final Resume–JD Fit Score: {final_fit_score}%")

    print(f"✅ Matched Skills: {matched_skills if matched_skills else 'None'}")
    print(f"❌ Missing Skills: {missing_skills if missing_skills else 'None'}")

    if missing_skills:
        print("\n📚 Recommended Courses for Missing Skills:")
        for skill, course in recommended_courses.items():
            print(f"  - {skill}: {course}")